# 📊 Modelagem de Banco de Dados com SQL

Bem-vindo ao seu primeiro contato com **SQL** (Structured Query Language)! 

## O que é um Banco de Dados?

Um **banco de dados** é como um arquivo Excel muito mais poderoso, onde podemos:
- Armazenar informações de forma organizada em **tabelas**
- Garantir que os dados estejam sempre consistentes
- Fazer consultas complexas rapidamente
- Controlar quem pode acessar cada informação

## O que é SQL?

**SQL** é a linguagem que usamos para "conversar" com bancos de dados. Com ela podemos:
- **CREATE**: Criar tabelas e estruturas
- **INSERT**: Adicionar dados
- **SELECT**: Buscar informações
- **UPDATE**: Modificar dados existentes
- **DELETE**: Remover dados

## Nossa Missão 🎯

Vamos criar um banco de dados para uma **empresa de consultoria**! Precisamos organizar informações sobre:
- 👥 **Clientes**: empresas que contratam nossos serviços
- 📋 **Projetos**: trabalhos que realizamos
- 👨‍💼 **Consultores**: nossa equipe
- 🔗 **Alocações**: quem trabalha em cada projeto
- ⭐ **Feedbacks**: avaliações dos projetos

## 🗝️ Conceitos Fundamentais: Chave Primária (PK) e Chave Estrangeira (FK)

### Chave Primária (Primary Key - PK)
É o **identificador único** de cada linha na tabela. Pense como o CPF de uma pessoa:
- Cada linha tem um ID único
- Nunca se repete
- Não pode ser vazio (NULL)

### Chave Estrangeira (Foreign Key - FK)
É uma **referência** que conecta duas tabelas. Como um "telefone de contato":
- Aponta para a PK de outra tabela
- Cria relacionamentos entre tabelas
- Mantém a consistência dos dados

### Exemplo de Relacionamento:
```
CLIENTES                    PROJETOS
+----+-------------+       +----+----------+------------+
| id | nome        |  ←──  | id | titulo   | cliente_id |
+----+-------------+       +----+----------+------------+
| 1  | TechCorp    |       | 1  | ERP I    | 1          |
| 2  | Verde Agro  |       | 2  | VENDAS II| 1          |
+----+-------------+       +----+----------+------------+
```

O `cliente_id` na tabela PROJETOS é uma FK que aponta para o `id` na tabela CLIENTES!

In [138]:
# Configuração inicial do banco de dados
import sqlite3
import os
import sys

# Adicionando o diretório de testes ao path
sys.path.append('tests')
from tests_modelagem import *

# Conectando ao banco de dados
conn = sqlite3.connect('consultoria.db')

# Se banco já existe, apaga tudo
conn.execute("DROP TABLE IF EXISTS projetos")
conn.execute("DROP TABLE IF EXISTS clientes")
conn.execute("DROP TABLE IF EXISTS consultores")
conn.execute("DROP TABLE IF EXISTS feedbacks")
conn.execute("DROP TABLE IF EXISTS alocacoes")

# Habilitando suporte a chaves estrangeiras (muito importante!)
conn.execute("PRAGMA foreign_keys = ON;")

print("✅ Conectado ao banco de dados 'consultoria.db'")
print("✅ Chaves estrangeiras habilitadas")

# Sistema de backup para rollback em caso de erro
backup_conn = None

def backup_db():
    """Cria um backup do banco em memória"""
    global backup_conn
    backup_conn = sqlite3.connect(':memory:')
    conn.backup(backup_conn)
    print("📁 Backup criado")

def restore_db():
    """Restaura o banco a partir do backup"""
    global backup_conn
    if backup_conn:
        backup_conn.backup(conn)
        print("🔄 Banco restaurado do backup")
    else:
        print("❌ Nenhum backup disponível")

def validate_and_execute(query, test_function):
    """Valida query em ambiente isolado antes de executar no banco principal"""
    if not query or not query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    # Fazer backup ANTES de qualquer tentativa
    backup_db()
    
    try:
        # Testar a query em um banco temporário isolado
        test_conn = sqlite3.connect(':memory:')
        test_conn.execute("PRAGMA foreign_keys = ON;")
        
        # Copiar estrutura existente para o banco de teste
        for linha in conn.iterdump():
            if not linha.startswith('BEGIN') and not linha.startswith('COMMIT'):
                try:
                    test_conn.execute(linha)
                except:
                    pass  # Ignora erros de estrutura já existente
        
        # Testar a query do usuário no banco isolado
        test_conn.execute(query)
        test_conn.commit()
        
        # Se chegou até aqui, a query é válida sintaticamente
        # Agora executar no banco principal
        conn.execute(query)
        conn.commit()
        
        # Testar se o resultado está correto
        success = test_function(conn)
        
        if success:
            print("✅ Query executada e validada com sucesso!")
            return True
        else:
            # Se teste falhar, fazer rollback
            restore_db()
            print("💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        print(f"❌ Erro na query: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False
    finally:
        if 'test_conn' in locals():
            test_conn.close()

def test_and_rollback(test_function):
    """Mantido para compatibilidade - executa teste simples"""
    success = test_function(conn)
    if not success:
        print("💡 Execute a query correta primeiro!")
    return success

def validate_and_execute_inserts(insert_queries, test_function):
    """Valida e executa múltiplas queries INSERT com rollback automático"""
    if not insert_queries or not insert_queries.strip():
        print("❌ Queries vazias! Escreva seus INSERTs antes de executar.")
        return False
    
    # Fazer backup ANTES de qualquer tentativa
    backup_db()
    
    try:
        # Executar todas as queries de inserção
        queries = [q.strip() for q in insert_queries.strip().split(';') if q.strip()]
        
        if not queries:
            print("❌ Nenhuma query válida encontrada!")
            return False
        
        for query in queries:
            conn.execute(query)
        conn.commit()
        
        # Testar se o resultado está correto
        success = test_function(conn)
        
        if success:
            print("✅ Dados inseridos e validados com sucesso!")
            return True
        else:
            # Se teste falhar, fazer rollback
            restore_db()
            print("💡 Dados inseridos mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        restore_db()
        print(f"❌ Erro ao inserir dados: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False

print("\n🛠️ Sistema de backup/restore configurado!")
print("Agora vamos começar a criar nossas tabelas...")

✅ Conectado ao banco de dados 'consultoria.db'
✅ Chaves estrangeiras habilitadas

🛠️ Sistema de backup/restore configurado!
Agora vamos começar a criar nossas tabelas...


## 🏗️ Criando Nossa Primeira Tabela: CLIENTES

### Sintaxe do CREATE TABLE:
```sql
CREATE TABLE nome_da_tabela (
    coluna1 TIPO RESTRICOES,
    coluna2 TIPO RESTRICOES,
    ...
);
```

### Tipos de Dados Comuns:
- **INTEGER**: números inteiros (1, 2, 3...)
- **TEXT**: texto ("João", "São Paulo")
- **REAL**: números decimais (10.5, 99.99)
- **DATE**: datas (2024-01-15)

### Restrições Importantes:
- **PRIMARY KEY**: define a chave primária
- **NOT NULL**: campo obrigatório
- **AUTOINCREMENT**: valor incrementa automaticamente

### Nossa Tabela CLIENTES:
- **id**: identificador único (PK, autoincrement)
- **nome**: nome da empresa (obrigatório)
- **setor**: setor de atuação (obrigatório)
- **cidade**: cidade onde fica (obrigatório)
- **uf**: estado com 2 letras (obrigatório)

In [139]:
# TODO: Crie a tabela CLIENTES
# Dica: Use a sintaxe CREATE TABLE com as colunas mencionadas acima
# Lembre-se: id deve ser PRIMARY KEY AUTOINCREMENT, outros campos NOT NULL

query_clientes = """
CREATE TABLE clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    setor TEXT NOT NULL,
    cidade TEXT NOT NULL,
    uf CHAR(2) NOT NULL
);

"""

# Quando terminar sua query, execute a validação:
validate_and_execute(query_clientes, test_create_clientes)

📁 Backup criado
✅ PASSOU: Criação da tabela clientes
✅ Query executada e validada com sucesso!


True

## 🔗 Criando as Demais Tabelas

Agora vamos criar as outras tabelas, prestando atenção nas **dependências**. 

### Ordem de Criação (IMPORTANTE!):
1. ✅ CLIENTES (já criada)
2. 🔄 PROJETOS (depende de CLIENTES)
3. 🔄 CONSULTORES (independente)
4. 🔄 ALOCACOES (depende de PROJETOS e CONSULTORES)
5. 🔄 FEEDBACKS (depende de PROJETOS)

### Por que a ordem importa?
Quando criamos uma FK (chave estrangeira), a tabela referenciada **já deve existir**!

### 📋 Tabela PROJETOS

Armazena os trabalhos que realizamos para os clientes.

**Colunas:**
- **id**: PK, autoincrement
- **titulo**: nome do projeto (obrigatório) - formato: "PALAVRA I", "PALAVRA II"
- **escopo**: descrição do que será feito
- **data_inicio**: quando começou (obrigatório)
- **data_fim**: quando terminou (pode ser NULL se ainda em andamento)
- **cliente_id**: FK → clientes.id (obrigatório)

**Restrições de FK:**
- ON DELETE RESTRICT: não pode apagar cliente se tem projetos
- ON UPDATE CASCADE: se ID do cliente mudar, atualiza aqui também

In [140]:
# TODO: Crie a tabela PROJETOS
# Dica: Use FOREIGN KEY (cliente_id) REFERENCES clientes(id) ON DELETE RESTRICT ON UPDATE CASCADE

query_projetos = """
CREATE TABLE projetos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL,
    escopo TEXT,
    data_inicio DATE NOT NULL,
    data_fim DATE,
    cliente_id INTEGER NOT NULL,
    FOREIGN KEY (cliente_id)
        REFERENCES CLIENTES(id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);

"""

# Quando terminar sua query, execute a validação:
validate_and_execute(query_projetos, test_create_projetos)

📁 Backup criado
✅ PASSOU: Criação da tabela projetos
✅ Query executada e validada com sucesso!


True

### 👨‍💼 Tabela CONSULTORES

Nossa equipe de trabalho, dividida por frentes de atuação.

**Colunas:**
- **id**: PK, autoincrement  
- **nome**: nome completo (obrigatório)
- **frente**: área de atuação (obrigatório) - valores: 'ENG', 'BUS', 'DIR'

In [141]:
# TODO: Crie a tabela CONSULTORES
query_consultores = """
CREATE TABLE consultores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    frente TEXT NOT NULL CHECK (frente IN ('ENG', 'BUS', 'DIR'))
);

"""

# Quando terminar sua query, execute a validação:
validate_and_execute(query_consultores, test_create_consultores)

📁 Backup criado
✅ PASSOU: Criação da tabela consultores
✅ Query executada e validada com sucesso!


True

### 🔗 Tabela ALOCACOES

Conecta consultores aos projetos, registrando quantas horas cada um trabalhou.

**Colunas:**
- **id**: PK, autoincrement
- **projeto_id**: FK → projetos.id (ON DELETE CASCADE, ON UPDATE CASCADE)
- **consultor_id**: FK → consultores.id (ON DELETE RESTRICT, ON UPDATE CASCADE)  
- **horas_trabalhadas**: REAL NOT NULL CHECK (>=0)

**Restrição especial:**
- UNIQUE(projeto_id, consultor_id): um consultor não pode ser alocado duas vezes no mesmo projeto

In [142]:
# TODO: Crie a tabela ALOCACOES
# Dica: Use CHECK para validar horas >= 0 e UNIQUE para evitar duplicatas
query_alocacoes = """
CREATE TABLE alocacoes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    projeto_id INTEGER NOT NULL,
    consultor_id INTEGER NOT NULL,
    horas_trabalhadas REAL NOT NULL CHECK (horas_trabalhadas >= 0),
    UNIQUE (projeto_id, consultor_id),
    FOREIGN KEY (projeto_id)
        REFERENCES PROJETOS(id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
    FOREIGN KEY (consultor_id)
        REFERENCES CONSULTORES(id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);

"""

# Quando terminar sua query, execute a validação:
validate_and_execute(query_alocacoes, test_create_alocacoes)

📁 Backup criado
✅ PASSOU: Criação da tabela alocacoes
✅ Query executada e validada com sucesso!


True

### ⭐ Tabela FEEDBACKS

Avaliações dos projetos realizados.

**Colunas:**
- **id**: PK, autoincrement
- **projeto_id**: FK → projetos.id (ON DELETE CASCADE, ON UPDATE CASCADE)
- **nota**: INTEGER NOT NULL CHECK (1 ≤ nota ≤ 5) 
- **comentario**: TEXT (opcional)

In [143]:
# TODO: Crie a tabela FEEDBACKS  
# Dica: Use CHECK para garantir nota entre 1 e 5
query_feedbacks = """
CREATE TABLE feedbacks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    projeto_id INTEGER NOT NULL,
    nota INTEGER NOT NULL CHECK (nota BETWEEN 1 AND 5),
    comentario TEXT,
    FOREIGN KEY (projeto_id)
        REFERENCES PROJETOS(id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);


"""

# Quando terminar sua query, execute a validação:
validate_and_execute(query_feedbacks, test_create_feedbacks)

📁 Backup criado
✅ PASSOU: Criação da tabela feedbacks
✅ Query executada e validada com sucesso!


True

## 📝 Inserindo Dados nas Tabelas

Agora que criamos a estrutura, vamos povoar nosso banco com dados!

### Sintaxe do INSERT:
```sql
INSERT INTO nome_tabela (coluna1, coluna2, coluna3) 
VALUES (valor1, valor2, valor3);
```

### Dicas importantes:
- **Strings** vão entre aspas simples: 'Texto aqui'
- **Datas** no formato: '2024-01-15'  
- **IDs** são gerados automaticamente (AUTOINCREMENT)
- **NULL** para valores vazios

### Ordem de inserção (por causa das FKs):
1. CLIENTES (não depende de ninguém)
2. CONSULTORES (não depende de ninguém)  
3. PROJETOS (precisa do cliente_id)
4. ALOCACOES (precisa de projeto_id e consultor_id)
5. FEEDBACKS (precisa do projeto_id)

In [144]:
# TODO: Insira 5 clientes
# Dados esperados:
# 0. NOME, SETOR, CIDADE, UF                
# 1. TechCorp Solutions, Tecnologia, São Paulo, SP
# 2. Verde Agro Ltda, Agronegócio, Campinas, SP  
# 3. MetalMax Indústrias, Metalurgia, Santos, SP
# 4. EduCare Ensino, Educação, Ribeirão Preto, SP
# 5. FastLogistic S.A., Logística, São José dos Campos, SP

insert_clientes = """
INSERT INTO clientes (nome, setor, cidade, uf) VALUES
('TechCorp Solutions', 'Tecnologia', 'São Paulo', 'SP'),
('Verde Agro Ltda', 'Agronegócio', 'Campinas', 'SP'),
('MetalMax Indústrias', 'Metalurgia', 'Santos', 'SP'),
('EduCare Ensino', 'Educação', 'Ribeirão Preto', 'SP'),
('FastLogistic S.A.', 'Logística', 'São José dos Campos', 'SP');
"""


# Quando terminar suas queries, execute a validação:
validate_and_execute_inserts(insert_clientes, test_insert_clientes)

📁 Backup criado
✅ PASSOU: Inserção de dados na tabela clientes
✅ Dados inseridos e validados com sucesso!


True

In [145]:
# TODO: Insira 5 consultores
# Dados esperados:
# 0. NOME, FRENTE
# 1. Ana Silva, ENG
# 2. Carlos Santos, BUS
# 3. Maria Oliveira, DIR
# 4. João Costa, ENG
# 5. Fernanda Lima, BUS

insert_consultores = """
INSERT INTO CONSULTORES (nome, frente) VALUES
('Ana Silva', 'ENG'),
('Carlos Santos', 'BUS'),
('Maria Oliveira', 'DIR'),
('João Costa', 'ENG'),
('Fernanda Lima', 'BUS');
"""


validate_and_execute_inserts(insert_consultores, test_insert_consultores)
print("💡 Descomente a linha acima quando terminar!")

📁 Backup criado
✅ PASSOU: Inserção de dados na tabela consultores
✅ Dados inseridos e validados com sucesso!
💡 Descomente a linha acima quando terminar!


In [146]:
# TODO: Insira 6 projetos (títulos no formato CAPS + numeral romano)
# Dados esperados:
# 0. TITULO, DESCRICAO, DATA_INICIO, DATA_FIM, CLIENTE_ID
# 1. ERP I, Implementação de sistema ERP, 2024-01-15, 2024-06-15, cliente_id=1
# 2. MARKETING II, Estratégia de marketing digital, 2024-02-01, 2024-08-01, cliente_id=2
# 3. LOGISTICA III, Otimização de processos logísticos, 2024-03-10, NULL, cliente_id=5
# 4. EDUCACAO IV, Plataforma de ensino online, 2024-04-05, 2024-10-05, cliente_id=4
# 5. VENDAS V, Automação de vendas, 2024-05-20, NULL, cliente_id=3
# 6. FINANCEIRO VI, Sistema financeiro integrado, 2024-06-15, NULL, cliente_id=1

insert_projetos = """
INSERT INTO PROJETOS (titulo, escopo, data_inicio, data_fim, cliente_id) VALUES
('ERP I', 'Implementação de sistema ERP', '2024-01-15', '2024-06-15', 1),
('MARKETING II', 'Estratégia de marketing digital', '2024-02-01', '2024-08-01', 2),
('LOGISTICA III', 'Otimização de processos logísticos', '2024-03-10', NULL, 5),
('EDUCACAO IV', 'Plataforma de ensino online', '2024-04-05', '2024-10-05', 4),
('VENDAS V', 'Automação de vendas', '2024-05-20', NULL, 3),
('FINANCEIRO VI', 'Sistema financeiro integrado', '2024-06-15', NULL, 1);
"""


validate_and_execute_inserts(insert_projetos, test_insert_projetos)
print("💡 Descomente a linha acima quando terminar!")

📁 Backup criado
✅ PASSOU: Inserção de dados na tabela projetos
✅ Dados inseridos e validados com sucesso!
💡 Descomente a linha acima quando terminar!


In [147]:
# TODO: Insira 10 alocações
# Dados esperados (projeto_id, consultor_id, horas):
# (1,1,120.5), (1,3,80.0), (2,2,95.5), (2,5,110.0), (3,1,150.0)
# (3,4,75.5), (4,5,200.0), (5,2,60.0), (5,3,85.5), (6,1,90.0)

insert_alocacoes = """
INSERT INTO ALOCACOES (projeto_id, consultor_id, horas_trabalhadas) VALUES
(1, 1, 120.5),
(1, 3, 80.0),
(2, 2, 95.5),
(2, 5, 110.0),
(3, 1, 150.0),
(3, 4, 75.5),
(4, 5, 200.0),
(5, 2, 60.0),
(5, 3, 85.5),
(6, 1, 90.0);
"""

validate_and_execute_inserts(insert_alocacoes, test_insert_alocacoes)
print("💡 Descomente a linha acima quando terminar!")

📁 Backup criado
✅ PASSOU: Inserção de dados na tabela alocacoes
✅ Dados inseridos e validados com sucesso!
💡 Descomente a linha acima quando terminar!


In [148]:
# TODO: Insira 5 feedbacks
# Dados esperados (projeto_id, nota, comentario):
# (1, 5, 'Excelente trabalho, superou expectativas')
# (2, 4, 'Bom resultado, algumas melhorias necessárias')
# (4, 5, 'Projeto entregue com qualidade excepcional')  
# (5, 3, 'Resultado satisfatório, pode melhorar')
# (6, 4, 'Boa execução dentro do prazo')

insert_feedbacks = """
INSERT INTO feedbacks (projeto_id, nota, comentario) VALUES
(1, 5, 'Excelente trabalho, superou expectativas'),
(2, 4, 'Bom resultado, algumas melhorias necessárias'),
(4, 5, 'Projeto entregue com qualidade excepcional'),
(5, 3, 'Resultado satisfatório, pode melhorar'),
(6, 4, 'Boa execução dentro do prazo');
"""


validate_and_execute_inserts(insert_feedbacks, test_insert_feedbacks)
print("💡 Descomente a linha acima quando terminar!")

📁 Backup criado
✅ PASSOU: Inserção de dados na tabela feedbacks
✅ Dados inseridos e validados com sucesso!
💡 Descomente a linha acima quando terminar!


## 🎉 Parabéns! Você Criou Seu Primeiro Banco de Dados!

### O que você aprendeu:

✅ **Conceitos fundamentais**:
- O que são bancos de dados e SQL
- Chaves primárias (PK) e estrangeiras (FK)
- Relacionamentos entre tabelas

✅ **Comandos DDL (Data Definition Language)**:
- `CREATE TABLE` para criar estruturas
- Tipos de dados (INTEGER, TEXT, REAL, DATE)
- Restrições (NOT NULL, CHECK, UNIQUE)

✅ **Comandos DML (Data Manipulation Language)**:
- `INSERT INTO` para adicionar dados
- Importância da ordem de inserção

✅ **Integridade referencial**:
- Como FK mantêm consistência
- ON DELETE e ON UPDATE

### Próximos passos:
Na próxima lição, aprenderemos consultas com `SELECT` para extrair informações do banco!

### Verificação final:
Execute a célula abaixo para ver um resumo do que foi criado:

In [149]:
# Verificação final - resumo do banco criado
import pandas as pd

print("📊 RESUMO DO BANCO DE DADOS CRIADO")
print("=" * 50)

# Checa se a conn esta ligada, se nao estiver, liga
conn.close()
conn = sqlite3.connect('consultoria.db')

# Lista as tabelas criadas
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print(f"\n🏗️  Tabelas criadas: {len(tables)}")
for table in tables:
    if (table[0] != 'sqlite_sequence'):  # Ignora tabela interna de sequência
        print(f"   • {table[0]}")

# Conta registros em cada tabela
print(f"\n📊 Registros por tabela:")
for table in tables:
    if (table[0] != 'sqlite_sequence'):  # Ignora tabela interna de sequência
        cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
        count = cursor.fetchone()[0]
        print(f"   • {table[0]}: {count} registros")

print(f"\n✅ Banco 'consultoria.db' criado com sucesso!")
print(f"🔗 Sistema de relacionamentos funcionando!")

# Fechando conexão
conn.close()
print(f"\n🔌 Conexão fechada. Exercício concluído!")

📊 RESUMO DO BANCO DE DADOS CRIADO

🏗️  Tabelas criadas: 6
   • clientes
   • projetos
   • consultores
   • alocacoes
   • feedbacks

📊 Registros por tabela:
   • clientes: 5 registros
   • projetos: 6 registros
   • consultores: 5 registros
   • alocacoes: 10 registros
   • feedbacks: 5 registros

✅ Banco 'consultoria.db' criado com sucesso!
🔗 Sistema de relacionamentos funcionando!

🔌 Conexão fechada. Exercício concluído!


In [134]:
# 🔄 IMPORTANTE: Recarregando módulo de testes
# Isso garante que novas funções adicionadas a test_join.py sejam reconhecidas

import sys
import importlib

# Adiciona o diretório tests ao path (se ainda não foi feito)
tests_path = 'tests'
if tests_path not in sys.path:
    sys.path.append(tests_path)

# Importa e recarrega o módulo para pegar as últimas mudanças
try:
    import test_join
    importlib.reload(test_join)
    print("✅ Módulo test_join recarregado com sucesso!")
    
    # Lista todas as funções disponíveis no módulo
    functions = [func for func in dir(test_join) if callable(getattr(test_join, func)) and not func.startswith('_')]
    print(f"📋 Funções disponíveis: {functions}")
    
    # Importa as funções específicas
    from test_join import test_inner_join, test_left_join,test_inner_join_medium,test_inner_join_hard
    print("✅ Funções test_inner_join e test_left_join importadas!")
    
except ImportError as e:
    print(f"❌ Erro ao importar test_join: {e}")
except AttributeError as e:
    print(f"❌ Erro: Uma das funções não foi encontrada: {e}")
    print("🔧 Verifique se test_inner_join e test_left_join estão definidas em test_join.py")

✅ Módulo test_join recarregado com sucesso!
📋 Funções disponíveis: ['test_inner_join', 'test_inner_join_hard', 'test_inner_join_medium', 'test_left_join']
✅ Funções test_inner_join e test_left_join importadas!
